In [2]:
import sys
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
model_dir= 'C:\\Users\\garav\\AGOP\\DLR\\trained_models\\CIFAR\\model2\\nn_models\\'
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import agop_gcnn as agc
from utils import trainer as tr
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
#import rfm
import numpy as np
from trained_models.CIFAR.model2 import trainer as t
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from utils.groupy.gconv.pytorch_gconv.splitgconv2d import P4ConvZ2, P4ConvP4, P4MConvZ2, P4MConvP4M
from groupy.gconv.make_gconv_indices import *
from copy import deepcopy
from torch.nn.functional import pad
from torch.func import jacrev

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


C:\Users\garav\AGOP\DLR\experiments\rfm_cifar
C:\Users\garav\AGOP\DLR
C:\Users\garav\AGOP\DLR\trained_models\CIFAR\model2\nn_models\


In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")


Using device: cuda:0


In [8]:
torch.cuda.empty_cache()

In [10]:
net= t.get_untrained_net()
print(net)
total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"Total number of trainable parameters in ResNet: {total_params}")

ResNet(
  (features): Sequential(
    (0): P4MConvZ2()
    (1): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): BasicBlock(
      (features): Sequential(
        (0): P4MConvP4M()
        (1): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): P4MConvP4M()
        (4): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
      (lrelu): ReLU()
    )
    (4): BasicBlock(
      (features): Sequential(
        (0): P4MConvP4M()
        (1): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): P4MConvP4M()
        (4): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
      (lrelu): ReLU()
    )
    (5): BasicBlock(
      (features): Sequential(
        (0): P4MConvP4M()
        (1)

In [12]:
trainloader, valloader, testloader = t.get_loaders()
#net= t.get_untrained_net()
init_net= deepcopy(net)
import os
if os.path.exists(model_dir+'cifar_gcnn_trained_nn.pth'):
    checkpoint = torch.load(model_dir+'cifar_gcnn_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")

#print("Train the network first")
#t.train_net()

  7%|█████▊                                                                        | 12.7M/170M [00:07<01:28, 1.77MB/s]

KeyboardInterrupt



In [ ]:
tr.visualize_predictions(net, testloader, range(10), device, num_images=36)